Import Libs

In [21]:
%matplotlib inline

import torch
import matplotlib.pyplot as plt
import requests
import os
import zipfile
import pandas as pd
from tqdm import tqdm
from numpy import isnan
from datetime import timedelta

Download Data

In [4]:
if not os.path.exists('data/electricity.zip'):
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip'
    response = requests.get(url, allow_redirects=True)

    with open("data/electricity.zip","wb") as handle:
        for data in tqdm(response.iter_content()):
            handle.write(data)

    with zipfile.ZipFile("data/electricity.zip",'r') as zip_ref:
        zip_ref.extractall("data")

    os.rename('data/household_power_consumption.txt','data/household_power_consumption.csv')


Preprocess Data

In [16]:
resolution = 60
df_raw = pd.read_csv('data/household_power_consumption.csv', sep=';', index_col=False, low_memory=False, na_values='?')
df_raw = df_raw.iloc[21996::resolution,:3]
df_raw['datetime'] = pd.to_datetime(df_raw['Date']+' '+df_raw['Time'])
df_raw = df_raw.drop(['Date','Time'],axis=1)
df_raw = df_raw.set_index(['datetime'])

In [36]:
g = df_raw.groupby(df_raw.index.floor('d'))

daily_data = []
my_day, final_day = df_raw.index[0].date(), df_raw.index[-1].date()

while my_day < final_day:
        day_data = g.get_group(my_day).values.tolist()
        if isnan(day_data).any():
            my_day += timedelta(days=1)
        else:
            row_data = [my_day.month, my_day.weekday()] + day_data
            daily_data.append(row_data)
            my_day += timedelta(days=1)

print('Dataset Size:'+str(daily_data.__len__()))

KeyError: datetime.date(2010, 1, 12)